In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests


In [1]:
import requests
import re
from IPython.display import display
import json

# Notebook on GitHub

Take the raw url of your notebook

In [2]:
raw_url = "https://raw.githubusercontent.com/jdh-observer/jdh001-JJszM3GwAYDs/main/namelesscrowd/The-Nameless-Crowds_V4.ipynb"
#raw_url = 'https://raw.githubusercontent.com/C2DH/jdh-notebook/master/author_guideline_template.ipynb'
r = requests.get(raw_url)
r.headers['content-type']
notebook = r.json()


# Local notebook

- Make a copy of your ipynb file and rename it on .json 
- Define the localPath variable

In [1]:
localPath = '/Users/elisabeth.guerard/c2dh/jdh-notebook/jdh001_0000_0002_0237_2532.json'
with open(localPath) as f:
    notebook = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/elisabeth.guerard/c2dh/jdh-notebook/jdh001_0000_0002_0237_2532.json'

# Generate statistics

Copy paste the result in the form https://journalofdigitalhistory.org/en/fingerprint

In [8]:
# use the cached response
cells = notebook.get('cells')
# output
cells_stats = []
# loop through cells and save relevant informations
for cell in cells:
    c = {
       'type': cell['cell_type'] 
    }
    # just skip if it's empty
    source = cell.get('source', [])
    if not source:
        continue
    # check cell metadata
    tags = cell.get('metadata').get('tags', [])
    if 'hidden' in tags:
        continue
    c['countLines'] = len(source)
    c['firstWords'] = ' '.join(source[0].split()[:5])
    c['countChars'] = len(''.join(source))
    c['tags'] = tags
    # c['source'] = cell.get('source')
    c['isMetadata'] = any(tag in ['title', 'abstract', 'contributor', 'disclaimer'] for tag in tags)
    c['isHermeneutic'] = any(tag in ['hermeneutics', 'hermeneutics-step'] for tag in tags)
    c['isFigure'] = any(tag.startswith('figure-') for tag in tags)
    c['isFullWidth'] = any(tag == 'full-width' for tag in tags)
    c['isHeading'] = cell['cell_type'] == 'markdown' and re.match(r'\s*#+\s', ''.join(cell.get('source'))) is not None
    cells_stats.append(c)

json.dumps({
    'cells': cells_stats
})

'{"cells": [{"type": "code", "countLines": 13, "firstWords": "%matplotlib inline", "countChars": 269, "tags": ["hermeneutics"], "isMetadata": false, "isHermeneutic": true, "isFigure": false, "isFullWidth": false, "isHeading": false}, {"type": "code", "countLines": 1, "firstWords": "noms_foule_df = pd.read_csv(\\"names-crowd_data.csv\\", sep=\\";\\", encoding=\\"latin1\\")", "countChars": 79, "tags": ["hermeneutics"], "isMetadata": false, "isHermeneutic": true, "isFigure": false, "isFullWidth": false, "isHeading": false}, {"type": "code", "countLines": 4, "firstWords": "# Using markdown while displaying", "countChars": 145, "tags": ["hermeneutics"], "isMetadata": false, "isHermeneutic": true, "isFigure": false, "isFullWidth": false, "isHeading": false}, {"type": "code", "countLines": 12, "firstWords": "import re", "countChars": 511, "tags": ["hermeneutics"], "isMetadata": false, "isHermeneutic": true, "isFigure": false, "isFullWidth": false, "isHeading": false}, {"type": "code", "countL

In [ ]:
json.dumps({
    'stats': {
        'countLines': sum([c['countLines'] for c in cells_stats]),
        'countChars': sum([c['countChars'] for c in cells_stats]),
        'countContributors': sum(['contributor' in c['tags'] for c in cells_stats]),
        'countHeadings': sum([c['isHeading'] for c in cells_stats]),
        'countHermeneuticCells': sum([c['isHermeneutic'] for c in cells_stats]),
        'countHermeneuticOnlyCells': sum(['hermeneutics' in c['tags'] for c in cells_stats]),
        'countCodeCells': sum([c['type'] == 'code' for c in cells_stats]),
        'countCells': len(cells_stats),
        'extentChars': [min([c['countChars'] for c in cells_stats]), max([c['countChars'] for c in cells_stats])],
        'extentLines': [min([c['countLines'] for c in cells_stats]), max([c['countLines'] for c in cells_stats])]
    },
    'cells': cells_stats
})